# YATS - Yet Another TSP Solution

## This notebook was created to serve a blog post by the same name.

In [1]:
# written in python 3.7.3
import numpy as np
import math
from geojson import LineString, Point, Feature, FeatureCollection
import geojsonio
import json
import random
random_seed = 42
random.seed(random_seed)

# Part I - The Code For Our Solution

### The Geographic Building Blocks

In [2]:
class GeoPoint:
    def __init__(self, lng: float, lat: float):
        # Why 5 digits? According to https://en.wikipedia.org/wiki/Decimal_degrees it's 1m. accuracy.
        self.lng = round(lng, 5)
        self.lat = round(lat, 5)
        
    def __repr__(self):
        # copy-pastable format to most map applications
        return f"[{self.lng}, {self.lat}]"
        
def euclidean_dist(geo_point_1: GeoPoint, geo_point_2: GeoPoint) -> float:
    d = np.linalg.norm(np.array([geo_point_1.lat, geo_point_1.lng])
                       - np.array([geo_point_2.lat, geo_point_2.lng]))
    return d

def get_geo_point_of_center(geo_points: [GeoPoint]) -> GeoPoint:
    lng_list, lat_list = list(zip(*[[g.lng, g.lat] for g in geo_points]))
    lng_center, lat_center = np.mean(np.array([lng_list, lat_list]), axis=1)
    return GeoPoint(lng_center, lat_center)

def get_angle_from_reference_geo_point_in_deg(reference_geo_point: GeoPoint, other_geo_point: GeoPoint) -> float:
    x = other_geo_point.lng - reference_geo_point.lng
    y = other_geo_point.lat - reference_geo_point.lat
    angle_from_reference_in_deg = math.degrees(math.atan2(y, x))
    return angle_from_reference_in_deg

### Our Initial Route - An Angular Route

In [3]:
def get_angular_route(geo_points: [GeoPoint]) -> [int]:
    center = get_geo_point_of_center(geo_points)
    route_idxs = sorted(list(range(len(geo_points))), 
                        key=lambda i: 
                        get_angle_from_reference_geo_point_in_deg(center, geo_points[i]),
                        reverse=True)
    return route_idxs

### Visualizing

In [4]:
def visualize(route_idxs: [int], geo_points: [GeoPoint]):
    lng_lat_list = [tuple([geo_points[i].lng, geo_points[i].lat])
                    for i in route_idxs]
    route = Feature(geometry=LineString(lng_lat_list),
                    properties={"name": "This is our route",
                                "stroke": "#8B0000"})
    places = [Feature(geometry=Point(lng_lat), 
                      properties={"name": f"Place {route_idxs[i]}",
                                  "marker-symbol": int(str(i)[-1]),
                                  "marker-color": "#00008B"})
              for i, lng_lat in enumerate(lng_lat_list)]
    
    feature_collection = FeatureCollection(features=[route] + places)
    geojsonio.display(json.dumps(feature_collection));

### Optimization step

In [5]:
def get_route_len(distances_array: np.array, route_idxs):
    route_len = sum([distances_array[i1][i2]
                     for i1, i2 in zip(route_idxs[:-1], route_idxs[1:])])
    return route_len

def optimize_route(distances_array: np.array, route_idxs: [int], n_iter: int) -> [int]:
    prev_cost = get_route_len(distances_array, route_idxs)
    
    all_idxs = list(range(len(route_idxs)))
    for _ in range(n_iter):    
        i1, i2 = random.sample(all_idxs, 2)
        route_idxs[i2], route_idxs[i1] = route_idxs[i1], route_idxs[i2]
        new_cost = get_route_len(distances_array, route_idxs)
        if new_cost < prev_cost:
            prev_cost = new_cost
        else:
            route_idxs[i2], route_idxs[i1] = route_idxs[i1], route_idxs[i2]
    return route_idxs

### Wrap It All Up

In [6]:
def plot_best_route(geo_points: [GeoPoint], distances_array: np.array, n_iter: int) -> None:
    route_idxs = get_angular_route(geo_points)
    route_idxs = optimize_route(distances_array, route_idxs, n_iter)
    visualize(route_idxs, geo_points)

# Part II - My Friday Morning Errands

In [7]:
rice = GeoPoint(34.904145, 32.178397)
veg = GeoPoint(34.899660, 32.178243)
pet= GeoPoint(34.899918, 32.177080)
garden = GeoPoint(34.904370, 32.173966)
pharm = GeoPoint(34.909027, 32.177480)
pasta = GeoPoint(34.906774, 32.178279)
pita = GeoPoint(34.903383, 32.177381)

geo_points = [rice, veg, pet, garden, pharm, pasta, pita]

In [8]:
distances_array = np.array([[0, 600, 550, 600, 650, 400, 150], 
                            [0, 0, 170, 900, 1100, 850, 550], 
                            [0, 0, 0, 750, 1000, 850, 400], 
                            [0, 0, 0, 0, 900, 750, 600], 
                            [0, 0, 0, 0, 0, 260, 700], 
                            [0, 0, 0, 0, 0, 0, 500], 
                            [0] * 7])
distances_array += distances_array.transpose()
distances_array

array([[   0,  600,  550,  600,  650,  400,  150],
       [ 600,    0,  170,  900, 1100,  850,  550],
       [ 550,  170,    0,  750, 1000,  850,  400],
       [ 600,  900,  750,    0,  900,  750,  600],
       [ 650, 1100, 1000,  900,    0,  260,  700],
       [ 400,  850,  850,  750,  260,    0,  500],
       [ 150,  550,  400,  600,  700,  500,    0]])

In [9]:
plot_best_route(geo_points, distances_array, 100)

# Part III - A Larger Experiment

In [10]:
def euclidean_dist(geo_point_1: GeoPoint, geo_point_2: GeoPoint):
    d = np.linalg.norm(np.array([geo_point_1.lat, geo_point_1.lng])
                       - np.array([geo_point_2.lat, geo_point_2.lng]))
    return d

In [11]:
n_points = 50
geo_points = [GeoPoint(34.8 + (0.1 * random.random()), 32.1 + (0.1 * random.random()))
              for _ in range(n_points)]
distances_array = np.array([[euclidean_dist(g_from, g_to)
                             for g_to in geo_points]
                            for g_from in geo_points])

In [12]:
%%time

n_iter = 10000
random_cost = get_route_len(distances_array, list(range(n_points)))
print(f"Route length for random order: {random_cost}")
route_idxs = get_angular_route(geo_points)
first_cost = get_route_len(distances_array, route_idxs)
print(f"Initial route length: {first_cost}")
route_idxs = optimize_route(distances_array, route_idxs, n_iter)
last_cost = get_route_len(distances_array, route_idxs)
print(f"Final route length: {last_cost}")
visualize(route_idxs, geo_points)

Route length for random order: 2.8800609543466313
Initial route length: 0.9013753942969249
Final route length: 0.6142705755858876
Wall time: 628 ms
